In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# functiont o scrape iitk webpage
def scrape_iitk_article(url):
    res = requests.get(url, headers ={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " +
                  "AppleWebKit/537.36 (KHTML, like Gecko) " +
                  "Chrome/114.0.0.0 Safari/537.36"
    })
    soup = BeautifulSoup(res.content, "html.parser")


    paragraphs = []
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if text:  # skip empty
            paragraphs.append(text)

    content = "\n".join(paragraphs)
    return content

In [ ]:
# extracting data from each link
links = ['https://iitk.ac.in/new/faculty-staff-campus-life', 'https://iitk.ac.in/new/iitk-history', 'https://iitk.ac.in/new/list-of-directors', 'https://iitk.ac.in/new/iitk-story', 'https://iitk.ac.in/new/the-fourth-iit-an-in-depth-analysis-of-iit-kanpur', 'https://iitk.ac.in/new/institute-overview', 'https://iitk.ac.in/new/education-at-iitk', 'https://iitk.ac.in/new/board-of-governers', 'https://iitk.ac.in/new/prof-manindra-agrawal', 'https://iitk.ac.in/new/key-administrators', 'https://iitk.ac.in/new/student-campus-life', 'https://www.iitk.ac.in/oir/', 'https://iitk.ac.in/new/kanpur-city', 'https://www.iitk.ac.in/wc/', 'https://iitk.ac.in/new/list-of-hobby-clubs', 'https://spo.iitk.ac.in/', 'https://iitk.ac.in/new/entrepreneurship-with-iitk', 'https://iitk.ac.in/new/academics-at-iit-kanpur', 'https://iitk.ac.in/new/innovation-highlights', 'https://iitk.ac.in/new/eligibility', 'https://iitk.ac.in/new/success-stories', 'https://iitk.ac.in/new/incubated-companies', 'https://www.iitk.ac.in/dora/the-foundation-board-directors', 'https://iitk.ac.in/new/culture-at-iitk', 'https://iitk.ac.in/new/cultural-events', 'https://iitk.ac.in/new/sports-at-iitk', 'https://iitk.ac.in/new/sports-facilities', 'https://iitk.ac.in/new/recreational-facilities', 'https://www.iitk.ac.in/aero/welcome-message-from-hod', 'https://www.iitk.ac.in/bsbe/', 'https://www.iitk.ac.in/che/', 'https://www.iitk.ac.in/ce/', 'https://www.cse.iitk.ac.in/', 'https://www.iitk.ac.in/design/', 'https://www.iitk.ac.in/ee/', 'https://www.iitk.ac.in/doms/', 'https://www.iitk.ac.in/mse/', 'https://www.iitk.ac.in/me/', 'https://iitk.ac.in/eco/', 'https://www.iitk.ac.in/chm/', 'https://www.iitk.ac.in/es/', 'https://www.iitk.ac.in/math/', 'https://www.iitk.ac.in/phy/', ]
big_chunks = []
for url in links:
    big_chunks.append(scrape_iitk_article(url))

In [4]:
print(len(big_chunks))

43


In [ ]:
# function to divide data into smaller chunks
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

def chunk_text(text, max_words=400, overlap=100):
    sentences = sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_word_count = 0

    i = 0
    while i < len(sentences):
        sentence = sentences[i]
        word_count = len(word_tokenize(sentence))

        if current_word_count + word_count <= max_words:
            current_chunk.append(sentence)
            current_word_count += word_count
            i += 1
        else:
            # Save chunk
            chunk_text = ' '.join(current_chunk).strip()
            chunks.append(chunk_text)

            # Move back overlap words
            if overlap > 0:
                backtrack_words = 0
                backtrack_index = len(current_chunk) - 1
                while backtrack_index >= 0 and backtrack_words < overlap:
                    backtrack_words += len(word_tokenize(current_chunk[backtrack_index]))
                    backtrack_index -= 1
                current_chunk = current_chunk[backtrack_index+1:]
                current_word_count = sum(len(word_tokenize(s)) for s in current_chunk)
            else:
                current_chunk = []
                current_word_count = 0

    # Save any remaining chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk).strip())

    return chunks

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shilp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\shilp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# breaking data to smaller chunks(processable ones)
from tqdm import tqdm
small_chunks = []
for chonkies in tqdm(big_chunks, desc = "Chunkifying"):
    for small_chonkies in chunk_text(chonkies):
        small_chunks.append(small_chonkies)

print(small_chunks[:5])

Chunkifying: 100%|██████████| 38/38 [00:00<00:00, 317.78it/s]

["beta\nIIT Kanpur offers a great environment for professional development. The beautiful, safe and well laid out campus has excellent facilities for recreation, children's education and healthcare. IITK offers a supportive enriching environment to its faculty and staff in all respects. The well laid out campus is a great place for children to grow up with excellent schools and a stimulating environment. Away from the hustle bustle of the city the IITK campus offers a peaceful and modern lifestyle to the faculty, staff and their families. There are several retail areas on the campus with shops and services to meet the day to day needs of the campus community. IITK has a faculty club along with several other formal and informal forums. There is great bonding among the campus community. The faculty and staff at IITK live on campus where they have easy access to facilities and activities. The great housing has contributed to the emergence of IITK as a leading institute. The comfortable an

In [29]:
print(len(small_chunks))

65


In [ ]:
# function to clean data
import unicodedata
import re

def clean_chunk(raw_html):
    # removing html code
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=" ") 
    # normalising unicode
    text = unicodedata.normalize("NFKC", text)

    # removing urls and emails
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)

    # fix bad spacing
    text = re.sub(r"\s+", " ", text)

    # fix punctuation
    text = re.sub(r"\.([A-Z])", r". \1", text)

    return text.strip()

In [ ]:
# cleaning data
for i in range(len(small_chunks)):
    small_chunks[i] = clean_chunk(small_chunks[i])

In [ ]:
# adding data to dataset
import json 
import os
if os.path.exists("dataset.json"):
    with open("dataset.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []

data.extend(small_chunks)

with open("dataset.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)